In [182]:
import pandas as pd
import numpy as np

In [169]:
df = pd.read_csv("./raw_data.csv",encoding="utf-8")

In [20]:
# geenrate dim table
# df2 = df.drop_duplicates()
# leader = pd.concat([df2["wla"],df2["wl1"],df2["wl2"],df2["wl3"]]).drop_duplicates()
# wonder = pd.concat([df2["ww1"],df2["ww2"],df2["ww3"],df2["ww4"],df2["ww5"],df2["ww6"]]).drop_duplicates()
# leader = leader[leader!='0'].rename("leader")
# wonder = wonder[wonder!='0'].rename("wonder")
# leader.to_csv('./leader.csv',index=False)
# wonder.to_csv('./wonder.csv',index=False)

## Player Win Rate

In [138]:
leader = pd.read_csv("./leader.csv",encoding="utf-8",index_col="leader")
wonder = pd.read_csv("./wonder.csv",encoding="utf-8",index_col="wonder")

In [189]:
def getLeaderResult(id):
    res = pd.DataFrame(columns=["id","card","win","lose","winrate"])
    df_win = df[df["player_win"]==id]
    df_lose = df[df["player_lose"]==id]
    for i,v in leader.iterrows():
        win = df_win[df_win["wl"+v[0]]==i].shape[0]
        lose = df_lose[df_lose["ll"+v[0]]==i].shape[0]
        res.loc[res.shape[0]] = [id,i,win,lose,win/(win+lose) if win+lose>0 else np.nan]
    return res

In [192]:
def getWonderResult(id):
    res = pd.DataFrame(columns=["id","card","win","lose","winrate"])
    df_win = df[df["player_win"]==id]
    df_lose = df[df["player_lose"]==id]
    for i,v in wonder.iterrows():
        win = df_win[(df_win["ww1"]==i)|(df_win["ww2"]==i)|(df_win["ww3"]==i)|(df_win["ww4"]==i)|(df_win["ww5"]==i)|(df_win["ww6"]==i)|(df_win["ww7"]==i)|(df_win["ww8"]==i)].shape[0]
        lose = df_lose[(df_lose["lw1"]==i)|(df_lose["lw2"]==i)|(df_lose["lw3"]==i)|(df_lose["lw4"]==i)|(df_lose["lw5"]==i)|(df_lose["lw6"]==i)|(df_lose["lw7"]==i)|(df_lose["lw8"]==i)].shape[0]
        res.loc[res.shape[0]] =  [id,i,win,lose,win/(win+lose) if win+lose>0 else np.nan]
    return res

In [194]:
df2 = getWonderResult("SuperDon")
df2.head()

,id,card,win,lose,winrate
0,SuperDon,雅典卫城,20,6,0.769231
1,SuperDon,大教堂,8,4,0.666667
2,SuperDon,马丘比丘,7,3,0.700000
3,SuperDon,空中花园,7,0,1.000000
4,SuperDon,姬路城,19,4,0.826087


In [ ]:
'''
import import_ipynb
from cov_leader import *
from cov_wonder import *

txt = open("5239.txt", "rt", encoding="utf-16")
s = txt.readlines()
txt.close()

txt = open("conbo.txt", "wt", encoding="utf-16")
for i in range(48+32):
    if i < 48:
        t1 = cov_num_to_leader(i)
    else:
        t1 = cov_num_to_wonder(i-48)
    for j in range(48+32):
        if j < 48:
            t2 = cov_num_to_leader(j)
        else:
            t2 = cov_num_to_wonder(j-48)
        leader_wonder = [t1, t2]
        txt.write(leader_wonder[0] + " " + leader_wonder[1] + " ")
        win = 0
        lose = 0
        for k in range(1, len(s)):
            s_s = s[k].split()
            if leader_wonder[0] in s_s[5:9] or leader_wonder[0] in s_s[13:21]:
                if leader_wonder[1] in s_s[5:9] or leader_wonder[1] in s_s[13:21]:
                    win += 1
            elif leader_wonder[0] in s_s[9:13] or leader_wonder[0] in s_s[21:]:
                if leader_wonder[1] in s_s[9:13] or leader_wonder[1] in s_s[21:]:
                    lose += 1

        txt.write(str(win) + " " + str(lose)+"\n")
'''

importing Jupyter notebook from cov_leader.ipynb
importing Jupyter notebook from cov_wonder.ipynb
